In [1]:
import pandas as pd
import glob
import os
import sys
import re
import time
start_time = time.time()

infol='tr_sub'

outname=infol+"_CpGdelta_info.txt"


In [2]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [3]:
allfiles=listdir_nohidden(infol)
allfiles

['tr_sub/g1_Tregs_g2_NaiveCD8',
 'tr_sub/g1_Tregs_g2_cm8',
 'tr_sub/g1_Tregs_g2_NaiveCD4']

In [4]:

def whichCTs(inputfile):
    m = re.search('g1_(.+?)_g2_(.+?)$', inputfile)
    if m:
        found1 = m.group(1)
        found2=m.group(2)

    else:
        print("found=",found)
        print("exiting")

        sys.exit(1)




    celltype1=found1
    celltype2=found2
    
    return celltype1,celltype2

In [5]:
filesdf=[]

for fp in allfiles:
    
    basefp=os.path.basename(fp)
    
    c1,c2=whichCTs(basefp)
    
    tmpdf=pd.read_csv(fp,sep="\t")
    #tmpdf['filename']=basefp
    
    tmpdf.rename(columns={c1+"-others": c1+"-"+c2},inplace=True,errors='raise')
    
    tmpdf=tmpdf[['chrom','start','end',c1+"-"+c2]]
    
    filesdf.append(tmpdf)
    

In [6]:
df_combined = pd.concat(filesdf, axis=0)
df_combined.set_index(['chrom','start','end'],inplace=True)
df_combined.head()

Tregs-NaiveCD8  Tregs-cm8  Tregs-NaiveCD4
chrom start  end                                              
chr1  686388 686390        0.333000        NaN             NaN
      687193 687195       -0.237500        NaN             NaN
      687328 687330       -0.323333        NaN             NaN
      922981 922983       -0.011167        NaN             NaN
      922991 922993       -0.045000        NaN             NaN

In [7]:
outdf=df_combined.groupby(df_combined.index).sum()
outdf.head()

,Tregs-NaiveCD8,Tregs-cm8,Tregs-NaiveCD4
"(chr1, 685875, 685877)",0.000000,-0.3330,-0.208000
"(chr1, 685999, 686001)",0.000000,0.0000,-0.043000
"(chr1, 686388, 686390)",0.333000,0.3330,0.041667
"(chr1, 687193, 687195)",-0.237500,0.0315,-0.153833
"(chr1, 687328, 687330)",-0.323333,0.0280,-0.347000


In [8]:
outdf.reset_index(inplace=True)

In [9]:
outdf[['chrom', 'start','end']] = pd.DataFrame(outdf['index'].tolist(), index=outdf.index)


In [10]:
outdf=outdf.drop(['index'],axis=1)

In [11]:
outdf=outdf.set_index(['chrom','start','end'])
outdf.shape

(11, 3)

In [12]:
outdf=outdf[(outdf != 0).all(axis=1)]
outdf.shape

(7, 3)

In [13]:
outdf.head()

Tregs-NaiveCD8  Tregs-cm8  Tregs-NaiveCD4
chrom start  end                                              
chr1  686388 686390        0.333000     0.3330        0.041667
      687193 687195       -0.237500     0.0315       -0.153833
      687328 687330       -0.323333     0.0280       -0.347000
      922981 922983       -0.011167    -0.0125       -0.023833
      922991 922993       -0.045000    -0.0305       -0.056000

In [14]:
scorecols=outdf.columns
scorecols

Index(['Tregs-NaiveCD8', 'Tregs-cm8', 'Tregs-NaiveCD4'], dtype='object')

In [15]:

outdf['maxdelta']=outdf[scorecols].max(axis=1)
outdf['mindelta']=outdf[scorecols].min(axis=1)
outdf['Avgdelta']=outdf[scorecols].mean(axis=1)

outdf.head()

Tregs-NaiveCD8  Tregs-cm8  Tregs-NaiveCD4  maxdelta  \
chrom start  end                                                           
chr1  686388 686390        0.333000     0.3330        0.041667  0.333000   
      687193 687195       -0.237500     0.0315       -0.153833  0.031500   
      687328 687330       -0.323333     0.0280       -0.347000  0.028000   
      922981 922983       -0.011167    -0.0125       -0.023833 -0.011167   
      922991 922993       -0.045000    -0.0305       -0.056000 -0.030500   

                     mindelta  Avgdelta  
chrom start  end                         
chr1  686388 686390  0.041667  0.235889  
      687193 687195 -0.237500 -0.119944  
      687328 687330 -0.347000 -0.214111  
      922981 922983 -0.023833 -0.015833  
      922991 922993 -0.056000 -0.043833

In [16]:
outdf.to_csv(outname,sep="\t")

In [17]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

0.17872905731201172
